In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import artpop

In [4]:
def sim_populations(metallicity, age_gyr):
    
    # Simulate population with ArtPop
    
    #######################################
    
    rng = np.random.RandomState(100) # random seed 
    band1 = 'ACS_WFC_F606W' 
    band2 = 'ACS_WFC_F814W'
    distance = 10 * u.pc # So that mags are in absolute magnitudes
    mass = (8 ** (9)) * u.Msun
    log_age = np.log10(float(age_gyr) * 10**(9))   
    
    #######################################    
    
    ssp = artpop.MISTSSP(
                         log_age = log_age,         # log of age in years
                         feh = metallicity,         # metallicity [Fe/H]
                         phot_system = 'HST_ACSWF', # photometric system(s)
                         total_mass = mass,         # stars
                         distance = distance,       # distance to system
                         mag_limit = -2,
                         mag_limit_band = 'ACS_WFC_F814W',
                         random_state = rng,         # random state for reproducibility
                         ab_or_vega = 'vega'
    )
    
    #######################################

    # Separate stellar sequences for plotting and isolating RGB to get "true" TRGB
    MS      = ssp.select_phase('MS')
    RGB     = ssp.select_phase('RGB')
    CHeB    = ssp.select_phase('CHeB')
    EAGB    = ssp.select_phase('EAGB')
    TPAGB   = ssp.select_phase('TPAGB')
    postAGB = ssp.select_phase('postAGB')
    
    # Turn into magnitudes
    mag1 = ssp.star_mags(band1) # F606W
    mag2 = ssp.star_mags(band2) # F814W
    
    #######################################
    
    # Get "true" TRGB as the brightest RGB star
    TRGB_true = np.min(mag2[RGB])
    
    # Get "true" contrast ratio based on the "true" TRGB
    idx_RGB = np.where((mag2 > TRGB_true) & (mag2 < TRGB_true + 0.5))[0] 
    idx_AGB = np.where((mag2 < TRGB_true) & (mag2 > TRGB_true - 0.5))[0] 
    R_true = len(idx_RGB) / max(len(idx_AGB), 1)

    #######################################
    
    return mag1, mag2, TRGB_true, R_true


In [5]:
sim_populations(-1, 1.288)

INFO: Fetching MIST synthetic photometry grid for HST_ACSWF.
INFO: Extracting grid from MIST_v1.2_vvcrit0.4_HST_ACSWF.txz.


(array([-1.27606503, -2.87880128, -3.59226949, ..., -1.34544094,
        -1.5070526 , -2.07888673]),
 array([-2.06362661, -3.86366546, -4.85874947, ..., -2.13999523,
        -2.31785398, -2.93149192]),
 -3.8818614806295324,
 7.632)